In [1]:
import pandas as pd
import sqlite3
from lets_plot import *
LetsPlot.setup_html()



In [2]:

conn = sqlite3.connect("vaaree_data.db")
query = "SELECT * FROM products " 
df = pd.read_sql(query, conn)
conn.close()
df.info()
df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  192 non-null    int64 
 1   Product_Name        192 non-null    object
 2   Material            192 non-null    object
 3   Price               192 non-null    int64 
 4   Product_Page        192 non-null    object
 5   MRP_Rate            192 non-null    int64 
 6   Offer               192 non-null    int64 
 7   Estimated_Delivery  192 non-null    object
 8   Category            192 non-null    object
 9   Rating              192 non-null    int64 
 10  Reviews             192 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 16.6+ KB


,ID,Product_Name,Material,Price,Product_Page,MRP_Rate,Offer,Estimated_Delivery,Category,Rating,Reviews
0,1388,Bloom Blush Planter - Set Of Two,Metal,799,https://vaaree.com/products/bloom-blush-plante...,2999,74,2025-03-10,Artificial Flowers & Plants,0,5
1,4099,(Coffee & White) Faux Pampas Grass Sticks (45....,Plastic,899,https://vaaree.com/products/faux-pampas-grass-...,3200,72,2025-03-10,Artificial Flowers & Plants,4,1
2,7283,Faux Realistic Bougainvillea Flower Stick - Pu...,Fabric,599,https://vaaree.com/products/faux-realistic-bou...,699,15,2025-03-03,Artificial Flowers & Plants,5,3
3,3552,Faux Tropic Anthurium Silk Plant With Pot - 2....,Pvc,959,https://vaaree.com/products/faux-tropic-anthur...,1750,46,2025-03-03,Artificial Flowers & Plants,3,3
4,4778,Goldo Glamour Planter - Set Of Two,Metal,699,https://vaaree.com/products/goldo-glamour-plan...,2699,75,2025-03-10,Artificial Flowers & Plants,0,3
...,...,...,...,...,...,...,...,...,...,...,...
187,9751,Memories To Rejoice Photo Frames (White) - Set...,Acrylic Wood,1335,https://vaaree.com/products/memories-to-rejoic...,4197,69,2025-03-10,Wall Decor,0,0
188,6572,Mayoora Regal Wall Art,Wood,899,https://vaaree.com/products/mayoora-regal-wall...,5000,82,2025-03-10,Wall Decor,1,1
189,4734,Hibiscus Bloom Wall Painting - Set Of Two,Canvas,1608,https://vaaree.com/products/hibiscus-bloom-wal...,5000,68,2025-03-10,Wall Decor,0,6
190,6408,Calm Buddha Wall Painting,Canvas,902,https://vaaree.com/products/calm-buddha-wall-p...,5000,82,2025-03-10,Wall Decor,0,2


In [3]:
df = df[df["Price"] != 0]

In [4]:
import os
save_path = r"D:\\dtproject\\Docs"
os.makedirs(save_path, exist_ok=True)

### 1. Why Some Products Have Low Offers vs. Others Have High Offers
##### Do high-rated products receive lower offers because of strong demand?
##### Are poorly reviewed products given higher discounts to increase sales?
##### Is there a link between offer percentage and review count?
 


In [5]:
#!pip3 install cairosvg


In [6]:
from lets_plot import ggsave, ggplot
import matplotlib.pyplot as plt
import os



df = df.copy()
df['Offer_Percent'] = ((df['MRP_Rate'] - df['Price']) / df['MRP_Rate']) * 100
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

bins = [-0.1, 3, 4, 5]
labels = ["Low (0 - 3)", "Medium (3 - 4)", "High (4 - 5)"]
df['Rating_Category'] = pd.cut(df['Rating'], bins=bins, labels=labels, include_lowest=True)
df = df.dropna(subset=['Rating_Category', 'Offer_Percent']).copy()

rating_offer_avg = df.groupby('Rating_Category', observed=False)['Offer_Percent'].mean()

if rating_offer_avg.get("High (4 - 5)", float('inf')) < rating_offer_avg.get("Low (0 - 3)", 0):
    print("\nHigh-rated products receive lower discounts, indicating strong demand.")
else:
    print("\nHigh-rated products do not necessarily receive lower discounts.")

p1 = (ggplot(df, aes(x='Rating_Category', y='Offer_Percent', fill='Rating_Category')) +
      geom_boxplot() +
      ggtitle("Offer Percentage Distribution by Rating Category") +
      xlab("Product Rating Category") +
      ylab("Average Offer Percentage (%)") +
      scale_x_discrete(labels=labels) +
      theme_minimal() +
      theme(legend_title=element_text(size=12),
            axis_title=element_text(size=14),
            axis_text=element_text(size=12)))
p1.show()



df['Reviews'] = pd.to_numeric(df['Reviews'], errors='coerce')
df = df.dropna(subset=['Reviews', 'Offer_Percent']).copy()

p2 = (ggplot(df, aes(x='Reviews', y='Offer_Percent')) +
      geom_point(color='blue', alpha=0.6) +
      ggtitle("Impact of Review Count on Offer Percentage") +
      xlab("Number of Reviews (Log Scale)") +
      ylab("Offer Percentage (%)") +
      scale_x_log10() +
      theme_minimal() +
      theme(legend_title=element_text(size=12),
            axis_title=element_text(size=14),
            axis_text=element_text(size=12)))
p2.show()


if len(df) > 1:
    correlation = df[['Offer_Percent', 'Reviews']].corr().iloc[0, 1]
    if correlation < 0:
        print("\nWeak negative correlation: Highly reviewed products tend to have lower discounts.")
    else:
        print("\nNo significant correlation between offer percentage and review count.")
else:
    print("\nNot enough data to compute correlation.")

ggsave(p1, filename=os.path.join(save_path, "Offer_Percentage_Distribution_by_Rating_Category.svg"),
       path=save_path, dpi=300)

ggsave(p2, filename=os.path.join(save_path, "Impact_of_Review_Count_on_Offer_Percentage.svg"),
       path=save_path)




High-rated products do not necessarily receive lower discounts.



Weak negative correlation: Highly reviewed products tend to have lower discounts.


'D:\\dtproject\\Docs\\Impact_of_Review_Count_on_Offer_Percentage.svg'

### 2. Categories with High Offers, Low Prices, High Ratings & Reviews
##### Identify categories where products are cheap but get high discounts, high ratings, and high reviews.
##### Are small decor items (e.g., table decor, lamps) more discounted than premium decor (e.g., sculptures, mirrors)?

In [7]:

df['Offer_Percent'] = ((df['MRP_Rate'] - df['Price']) / df['MRP_Rate']) * 100

category_analysis = df.groupby('Category').agg({'Offer_Percent': 'mean', 'Price': 'mean', 'Rating': 'mean', 'Reviews': 'mean'}).reset_index()

filtered_categories = category_analysis[(category_analysis['Offer_Percent'] > category_analysis['Offer_Percent'].median()) &
                                        (category_analysis['Price'] < category_analysis['Price'].median()) &
                                        (category_analysis['Rating'] > category_analysis['Rating'].median()) &
                                        (category_analysis['Reviews'] > category_analysis['Reviews'].median())]

print("\nCategories with High Offers, Low Prices, High Ratings & Reviews:")
print(filtered_categories.sort_values(by=['Offer_Percent', 'Price', 'Rating', 'Reviews'], ascending=[False, True, False, False]))

plot1 = ggplot(filtered_categories, aes(x='Category', y='Offer_Percent', fill='Category')) + \
        geom_bar(stat='identity') + \
        ggtitle("Categories with High Offers, Low Prices, High Ratings & Reviews") + \
        xlab("Category") + ylab("Average Offer Percentage") + \
        theme(axis_text_x=element_text(angle=45, hjust=1))
plot1.show()
ggsave(plot1, filename=os.path.join(save_path, "Categories with High Offers, Low Prices, High Ratings & Reviews.svg"),
       path=save_path, dpi=300)


print("\nConclusion:")
if not filtered_categories.empty:
    print("The following categories have products that are cheaper, receive high discounts, and maintain strong ratings and reviews:")
    for category in filtered_categories['Category']:
        print(f"- {category}")
else:
    print("No categories meet all the specified criteria simultaneously.")

small_decor = ["Artificial Flowers & Plants", "Candle Stand & Holders", "Clocks", "Lamps & Lighting"]
premium_decor = ["Mirrors", "Religious & Spiritual Items", "Showpieces, Vases & Accent Bowls", "Wall Decor"]

df['Decor_Type'] = df['Category'].apply(lambda x: 'Small Decor' if x in small_decor else ('Premium Decor' if x in premium_decor else 'Other'))

decor_analysis = df.groupby('Decor_Type')['Offer_Percent'].mean()


print("\nConclusion:")
if 'Small Decor' in decor_analysis.index and 'Premium Decor' in decor_analysis.index:
    if decor_analysis['Small Decor'] > decor_analysis['Premium Decor']:
        print("Small decor items receive higher discounts compared to premium decor items.")
    else:
        print("Premium decor items receive similar or higher discounts compared to small decor items.")
else:
    print("Not enough data for comparison.")

plot2 = ggplot(df, aes(x='Decor_Type', y='Offer_Percent', fill='Decor_Type')) + \
        geom_boxplot() + \
        ggtitle("Offer Percentage: Small Decor vs. Premium Decor") + \
        xlab("Decor Type") + ylab("Offer Percentage")
plot2.show()

ggsave(plot2, filename=os.path.join(save_path, "Offer_Percentage_Small_Decor_vs.Premium_Decor.svg"),
       path=save_path)


Categories with High Offers, Low Prices, High Ratings & Reviews:
     Category  Offer_Percent       Price    Rating   Reviews
7  Wall Decor      54.414681  926.791667  2.083333  6.333333



Conclusion:
The following categories have products that are cheaper, receive high discounts, and maintain strong ratings and reviews:
- Wall Decor

Conclusion:
Premium decor items receive similar or higher discounts compared to small decor items.


'D:\\dtproject\\Docs\\Offer_Percentage_Small_Decor_vs.Premium_Decor.svg'

### 3. Material Distribution Across Categories
##### Which materials are dominant in each category?
##### Are certain materials exclusive to certain categories (e.g., wood for furniture, glass for vases)?


In [8]:


material_distribution = df.groupby(['Category', 'Material']).size().reset_index(name='Count')

dominant_materials = material_distribution.loc[material_distribution.groupby('Category')['Count'].idxmax()]
print("\nDominant Materials in Each Category:")
print(dominant_materials)

exclusive_materials = material_distribution.groupby('Material')['Category'].nunique()
exclusive_materials = exclusive_materials[exclusive_materials == 1].index.tolist()
exclusive_df = material_distribution[material_distribution['Material'].isin(exclusive_materials)]
print("\nMaterials Exclusive to Certain Categories:")
print(exclusive_df)

print("\nConclusion:")
if not dominant_materials.empty:
    print("Each category has dominant materials, indicating material preferences for different product types.")
if not exclusive_df.empty:
    print("Certain materials are exclusive to specific categories, reinforcing material specialization in product design.")
else:
    print("No strictly exclusive materials found; many materials are used across multiple categories.")

from lets_plot import *
LetsPlot.setup_html()


material_distribution = material_distribution.sort_values(by='Count', ascending=False)


plot3 = (ggplot(material_distribution) +
         geom_bar(aes(x='Category', y='Count', fill='Material'), 
                  stat='identity', position='dodge', width=4) + 
         ggtitle("Material Distribution Across Categories") +
         xlab("Category") + ylab("Count") +
         theme_minimal() +
         theme(axis_text_x=element_text(angle=45, hjust=1, size=14),  
               axis_text_y=element_text(size=12),
               axis_title=element_text(size=16, face="bold"),
               plot_title=element_text(size=20, face="bold")) +
         scale_fill_brewer(type='qual', palette='Set2') +
         ggsize(1000, 550))  


plot3.show()
ggsave(plot3, filename=os.path.join(save_path, "Material Distribution Across Categories.svg"),
       path=save_path, dpi=300)




Dominant Materials in Each Category:
                            Category      Material  Count
5        Artificial Flowers & Plants         Metal      5
13            Candle Stand & Holders          Iron      7
23                            Clocks       Plastic     12
25                  Lamps & Lighting  Fabric Shade     14
33                           Mirrors          Iron     14
43       Religious & Spiritual Items         Metal      5
50  Showpieces, Vases & Accent Bowls          Iron      7
60                        Wall Decor           Mdf      6

Materials Exclusive to Certain Categories:
                            Category                Material  Count
0        Artificial Flowers & Plants                    Cane      1
3        Artificial Flowers & Plants  Dried Flowers & Leaves      2
4        Artificial Flowers & Plants                  Fabric      1
7        Artificial Flowers & Plants               Polyester      1
8        Artificial Flowers & Plants                    

'D:\\dtproject\\Docs\\Material Distribution Across Categories.svg'

### 5. Best Deals & Overpriced Items
##### Identify the top 10 best-value deals (low price, high discount, high rating).
##### Identify the top 10 overpriced items (high MRP, low offer, low reviews).


In [9]:

df['Offer_Percent'] = ((df['MRP_Rate'] - df['Price']) / df['MRP_Rate']) * 100


best_deals = df[(df['Price'] < df['Price'].median()) & 
                (df['Offer_Percent'] > df['Offer_Percent'].median()) & 
                (df['Rating'] > df['Rating'].median())&
                (df['Material'].notna())]

best_deals = best_deals.sort_values(by=['Offer_Percent', 'Rating'], ascending=[False, False]).head(10)

print("\nTop 10 Best Deals (Low Price, High Discount, High Rating):")
display(best_deals[['Product_Name', 'Material','Category', 'Price', 'Offer_Percent', 'Rating', 'Reviews']])


overpriced_items = df[(df['MRP_Rate'] > df['MRP_Rate'].median()) & 
                      (df['Offer_Percent'] < df['Offer_Percent'].median()) & 
                      (df['Reviews'] < df['Reviews'].median())]

overpriced_items = overpriced_items.sort_values(by=['MRP_Rate', 'Offer_Percent', 'Reviews'], ascending=[False, True, True]).head(10)

print("\nTop 10 Overpriced Items (High MRP, Low Offer, Low Reviews):")
display(overpriced_items[['Product_Name', 'Material','Category', 'MRP_Rate', 'Price', 'Offer_Percent', 'Reviews']])

plot_best_deals = (ggplot(best_deals, aes(x='Product_Name', y='Offer_Percent', fill='Category')) +
      geom_bar(stat='identity') +
      coord_flip() +
      ggtitle("🔥 Top 10 Best Deals: High Discount & Ratings") +
      xlab("Product") + ylab("Offer Percentage (%)") +
      theme_minimal() +
      ggsize(2000, 900))

plot_best_deals.show()
ggsave(plot_best_deals, filename=os.path.join(save_path, "Top 10 Best DealsHigh Discount & Ratings.svg"),
       path=save_path, dpi=300)


plot_overpriced = (ggplot(overpriced_items, aes(x='Product_Name', y='MRP_Rate', fill='Category')) +
      geom_bar(stat='identity') +
      coord_flip() +
      ggtitle("⚠️ Top 10 Overpriced Items: High MRP & Low Offer") +
      xlab("Product") + ylab("MRP Rate") +
      theme_minimal() +
      ggsize(900, 500))

plot_overpriced.show()
ggsave(plot_overpriced, filename=os.path.join(save_path, "Top 10 Overpriced Items High MRP & Low Offer.svg"),
       path=save_path, dpi=300)



Top 10 Best Deals (Low Price, High Discount, High Rating):


,Product_Name,Material,Category,Price,Offer_Percent,Rating,Reviews
25,Little Budha Smoke Fountain - Blue,Polyresin,Candle Stand & Holders,280,90.663555,1,3
121,Little Budha Smoke Fountain - Blue,Polyresin,Religious & Spiritual Items,280,90.663555,1,3
20,Druva Ethnic Planter - Blue & Yellow,Pvc,Artificial Flowers & Plants,202,79.800000,5,2
167,Sweethearts Showpiece - Gold,Aluminum,"Showpieces, Vases & Accent Bowls",795,75.000000,5,1
160,Spunky Splash Vase - White,Ceramic,"Showpieces, Vases & Accent Bowls",770,73.448276,5,1
177,Shyam Art Painting - Set Of Two,Wood,Wall Decor,540,73.000000,4,2
186,Acra Wall Accent,Mdf,Wall Decor,544,72.800000,3,1
16,Sofia Macrame Plant Hanger - Set Of Two,Dried Flowers & Leaves,Artificial Flowers & Plants,384,72.571429,4,3
149,Whimsy Paperbag Vase - Blue,Ceramic,"Showpieces, Vases & Accent Bowls",564,71.800000,5,2
34,Shri Ganesha And Lakshmi Tealight Candle Holde...,Metal,Candle Stand & Holders,260,71.078977,3,3



Top 10 Overpriced Items (High MRP, Low Offer, Low Reviews):


,Product_Name,Material,Category,MRP_Rate,Price,Offer_Percent,Reviews
116,Hexaga Fuse Mirror,Iron,Mirrors,5999,4999,16.669445,1
117,Piko Ethnic Wall Mirror,Iron,Mirrors,5999,4999,16.669445,1
69,Alice Table Clock,Aluminum,Clocks,5280,4400,16.666667,1
13,Faux Realistic Fiddle Leaf Fig Plant With Blac...,Dried Flowers & Leaves,Artificial Flowers & Plants,4499,3999,11.113581,1
166,Master Of Sea Showpiece,Metal,"Showpieces, Vases & Accent Bowls",3999,2399,40.010003,1
123,Shubhlabh Pooja Thali Set,German Silver,Religious & Spiritual Items,3938,2625,33.341798,2
54,Calypso Table Clock - Black,Aluminum,Clocks,3125,2325,25.600000,1
60,Leonay Table Clock,Aluminum,Clocks,3000,2232,25.600000,1
61,Ondene Flower Wall Clock,Iron,Clocks,2849,1890,33.660934,1
64,Sireen Wall Clock,Iron,Clocks,2835,1890,33.333333,1


'D:\\dtproject\\Docs\\Top 10 Overpriced Items High MRP & Low Offer.svg'

In [10]:

import numpy as np

### 6. Product Clustering Based on Pricing & Offers
##### Can we cluster products into segments like "Luxury," "Budget," and "Mid-range" based on pricing and discount patterns?



In [11]:


df['Offer_Percent'] = ((df['MRP_Rate'] - df['Price']) / df['MRP_Rate']) * 100

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['Price', 'Offer_Percent'], inplace=True)

price_bins = [df['Price'].min(), df['Price'].quantile(0.33), df['Price'].quantile(0.67), df['Price'].max()]
offer_bins = [df['Offer_Percent'].min(), df['Offer_Percent'].quantile(0.33), df['Offer_Percent'].quantile(0.67), df['Offer_Percent'].max()]

df['Price_Segment'] = pd.cut(df['Price'], bins=price_bins, labels=['Budget', 'Mid-range', 'Luxury'], include_lowest=True)
df['Offer_Segment'] = pd.cut(df['Offer_Percent'], bins=offer_bins, labels=['Low Offer', 'Medium Offer', 'High Offer'], include_lowest=True)
df['Segment'] = df['Price_Segment'].astype(str) + " - " + df['Offer_Segment'].astype(str)

print("\nProduct Segments Based on Pricing & Offers:")
display(df[['Product_Name', 'Category', 'Price', 'Offer_Percent', 'Segment']].head(20))

plot_clusters = ggplot(df, aes(x='Price', y='Offer_Percent', color='Segment')) + \
    geom_point(size=3, alpha=0.7) + \
    ggtitle("Product Segmentation: Luxury vs. Mid-range vs. Budget") + \
    xlab("Price") + ylab("Offer Percentage") + \
    theme_minimal()

plot_clusters.show()
ggsave(plot_clusters, filename=os.path.join(save_path, "Product Segmentation Luxury vs. Mid-range vs. Budget.svg"),
       path=save_path, dpi=300)



Product Segments Based on Pricing & Offers:


,Product_Name,Category,Price,Offer_Percent,Segment
0,Bloom Blush Planter - Set Of Two,Artificial Flowers & Plants,799,73.357786,Mid-range - High Offer
1,(Coffee & White) Faux Pampas Grass Sticks (45....,Artificial Flowers & Plants,899,71.906250,Mid-range - High Offer
2,Faux Realistic Bougainvillea Flower Stick - Pu...,Artificial Flowers & Plants,599,14.306152,Budget - Low Offer
3,Faux Tropic Anthurium Silk Plant With Pot - 2....,Artificial Flowers & Plants,959,45.200000,Mid-range - Medium Offer
4,Goldo Glamour Planter - Set Of Two,Artificial Flowers & Plants,699,74.101519,Budget - High Offer
5,Pebble Play Vase - White,Artificial Flowers & Plants,398,56.692057,Budget - Medium Offer
6,Paradise Finesse Planter - Set Of Two,Artificial Flowers & Plants,799,73.357786,Mid-range - High Offer
7,Artificial White Baby's Breath - Set Of Three,Artificial Flowers & Plants,600,70.000000,Budget - High Offer
8,Dalfol Solawood Floral Basket,Artificial Flowers & Plants,1910,36.120401,Luxury - Medium Offer
9,Faux Green Palm Plant With Pot - 2.62 ft,Artificial Flowers & Plants,1099,26.733333,Mid-range - Low Offer


'D:\\dtproject\\Docs\\Product Segmentation Luxury vs. Mid-range vs. Budget.svg'

In [12]:

conn = sqlite3.connect("vaaree_data.db")
query = "SELECT * FROM products " 
df = pd.read_sql(query, conn)
conn.close()



### 7. **Analysis of Set vs. Individual Products in Decoration Data**

#### Offer Comparison (Set vs Individual)
#### Category-wise Set Count
#### Material-wise Set Count
#### Top 10 Set Products with High MRP but Low Discount
#### Offer Comparison for High Rating Products (>=4) vs. Low Rating Products
#### Best Deals - Low Price, High Offer, High Rating
#### Categories with High Offer, Low Price, and High Ratings
#### Low Price, High Offer Products

In [13]:
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()

df["is_set"] = df["Product_Name"].str.contains("set of", case=False, na=False)

offer_comparison = df.groupby("is_set")["Offer"].mean()
category_set_count = df.groupby("Category")["is_set"].sum()
material_set_count = df.groupby("Material")["is_set"].sum().sort_values(ascending=False)

df["MRP_vs_Offer"] = df["MRP_Rate"] / df["Offer"]
high_mrp_low_discount = df[df["is_set"]].sort_values(by="MRP_vs_Offer", ascending=False).head(10)

df["high_rating"] = df["Rating"] >= 4 
rating_vs_offer = df.groupby(["high_rating", "is_set"]) ["Offer"].mean()

category_high_offer_low_price = df[(df["Price"] < df["Price"].median()) & (df["Offer"] > df["Offer"].median())]

low_price_high_offer = df[(df["Price"] < df["Price"].quantile(0.25)) & (df["Offer"] > df["Offer"].quantile(0.75))].nlargest(10, "Offer")[["Product_Name", "MRP_Rate", "Price", "Offer", "Category", "Rating"]]

offer_comparison = df.groupby("is_set")["Offer"].mean()
rating_vs_offer = df.groupby(["high_rating", "is_set"])["Offer"].mean()
material_set_count = df.groupby("Material")["is_set"].sum().sort_values(ascending=False)

plot1 = (ggplot(pd.DataFrame(offer_comparison).reset_index(), aes(x='is_set', y='Offer', fill='is_set')) +
         geom_bar(stat='identity') +
         ggtitle("Offer Comparison: Set vs Individual") +
         xlab("Is Set") + ylab("Average Offer Percentage"))
plot1.show()
ggsave(plot1, filename=os.path.join(save_path, "Offer Comparison Set vs Individual.svg"),
       path=save_path, dpi=300)

plot2 = (ggplot(pd.DataFrame(category_set_count).reset_index(), aes(x='Category', y='is_set', fill='Category')) +
         geom_bar(stat='identity') +
         ggtitle("Set Count by Category") +
         coord_flip())
plot2.show()
ggsave(plot2, filename=os.path.join(save_path, "Set Count by Category.svg"),
       path=save_path, dpi=300)


print("\nCategory-wise Set Count:")
display(df.groupby("Category")["is_set"].sum())
print("Material-wise Set Count:")
display(material_set_count)
print("\nLow Price, High Offer Top 10 Products:")
display(low_price_high_offer.head(10))
print("Top 10 Set Products with High MRP but Low Discount:")
display(high_mrp_low_discount[["Product_Name", "MRP_Rate", "Price", "Offer"]])
print("\nBest Deals - High Offer & High Rating Products:")
display(df[df['high_rating']][["Product_Name", "MRP_Rate", "Price", "Offer", "Rating", "Category"]])



Category-wise Set Count:


Category
Artificial Flowers & Plants         10
Candle Stand & Holders              13
Clocks                               0
Lamps & Lighting                     1
Mirrors                              4
Religious & Spiritual Items          7
Showpieces, Vases & Accent Bowls    12
Wall Decor                          13
Name: is_set, dtype: int64

Material-wise Set Count:


Material
Iron                      17
Metal                      8
Wood                       8
Mdf                        5
Polyresin                  3
Ceramic                    3
Acrylic Wood               2
Unknown                    2
Plastic                    2
Polyester                  1
Glass Shade                1
Canvas                     1
Dried Flowers & Leaves     1
Glass                      1
Mango Wood                 1
Cotton                     1
Silk                       1
Terracotta                 1
Resin                      1
Foam                       0
Aluminum                   0
Cane                       0
Brass                      0
German Silver              0
Fabric                     0
Fabric Shade               0
Marble                     0
Metal Shade                0
Papier Mache               0
Rattan Shade               0
Pvc                        0
Rattan                     0
Name: is_set, dtype: int64


Low Price, High Offer Top 10 Products:


,Product_Name,MRP_Rate,Price,Offer,Category,Rating
25,Little Budha Smoke Fountain - Blue,2999,280,91,Candle Stand & Holders,1
37,Little Budha Smoke Fountain - Yellow,2999,280,91,Candle Stand & Holders,0
47,Baby Buddha Monk Smoke Fountain - Green & Brown,2999,280,91,Candle Stand & Holders,0
121,Little Budha Smoke Fountain - Blue,2999,280,91,Religious & Spiritual Items,1
31,Lord Ganesha Smoke Fountain,2999,340,89,Candle Stand & Holders,0
135,Lord Ganesha Smoke Fountain,2999,340,89,Religious & Spiritual Items,0
151,Baby Monk Showpiece,1299,260,80,"Showpieces, Vases & Accent Bowls",0
10,Edelle Rose Floral Stick,1000,256,75,Artificial Flowers & Plants,0
33,Lantern Lore Tealight Candle Holder - White,999,260,74,Candle Stand & Holders,0
140,Lantern Lore Tealight Candle Holder - White,999,260,74,Religious & Spiritual Items,0


Top 10 Set Products with High MRP but Low Discount:


,Product_Name,MRP_Rate,Price,Offer
104,Aeliana Wall Mirror - Set Of Three,11999,5556,54
14,Naturally Dried Pampas Stems (Brown) - Set Of Ten,1642,1368,17
45,Kaede Candle Stand - Set Of Three,1597,1331,17
173,Vivid Indian Heritage Wall Art - Set Of Seven,7000,1340,81
171,Memories To Rejoice Photo Frames (Brown) - Set...,5995,1798,70
189,Hibiscus Bloom Wall Painting - Set Of Two,5000,1608,68
158,Melrose Tribal Terracotta Vase - Set Of Three,1849,1385,26
102,Jeneva Wall Mirror - Set Of Two,2331,1554,34
24,Oceane Wooden Tall Candle Stand - Set Of Two,4800,1299,73
120,Oceane Wooden Tall Candle Stand - Set Of Two,4800,1299,73



Best Deals - High Offer & High Rating Products:


,Product_Name,MRP_Rate,Price,Offer,Rating,Category
1,(Coffee & White) Faux Pampas Grass Sticks (45....,3200,899,72,4,Artificial Flowers & Plants
2,Faux Realistic Bougainvillea Flower Stick - Pu...,699,599,15,5,Artificial Flowers & Plants
12,Bit-Daisy Floral Stick - White,1000,0,70,4,Artificial Flowers & Plants
13,Faux Realistic Fiddle Leaf Fig Plant With Blac...,4499,3999,12,5,Artificial Flowers & Plants
15,Faux Banana Bonsai In Ceramic Pot (24 cms) - S...,2250,917,60,4,Artificial Flowers & Plants
16,Sofia Macrame Plant Hanger - Set Of Two,1400,384,73,4,Artificial Flowers & Plants
17,Elitta Ethnic Handcrafted Planter - Set Of Three,3199,896,72,5,Artificial Flowers & Plants
19,Faux Everlasting White Iceton Croton Plant Wit...,639,899,69,5,Artificial Flowers & Plants
20,Druva Ethnic Planter - Blue & Yellow,1000,202,70,5,Artificial Flowers & Plants
21,Lilium Roose Floral Sticks - Pink,1750,0,29,5,Artificial Flowers & Plants
